## 0. 导入依赖

In [1]:
from configparser import ConfigParser          # 读取 config.ini
from pathlib import Path                       # 处理路径
from pickle import dump                        # 保存构建好的 MicroCorpus

import sys
import os
sys.path.append(os.path.abspath(".."))  # 从 notebooks/ 回到外面的 MiCoGPT 根目录
from MiCoGPT.utils.corpus import MicroCorpus
from MiCoGPT.utils.base import CustomUnpickler

# 设置根目录
project_root = Path("/Users/kiancai/STA24/CWD/STAi/MiCoGPT")

# 生成路径
input_path = project_root / "data" / "try2_withCC/abundance_A_subgroup_55575.csv"
output_path = project_root / "data" / "try2_withCC/abundance_A_subgroup_55575.pkl"
config_path = project_root / "MiCoGPT" / "resources" / "mgm_config.ini"
tokenizer_path = project_root / "MiCoGPT" / "resources" / "MiCoGPTokenizer.pkl"

## 1. 定义 construct_simple 函数

In [2]:
def construct_simple(
    input_path: str,       # 输入丰度表文件路径（h5 / csv / tsv / txt）
    output_path: str,      # 输出的 MicroCorpus pkl 路径
    config_path: str,      # config.ini 的路径
    tokenizer_path: str,   # MicroTokenizer.pkl 的路径
    key: str = "genus",
    no_normalize: bool = False,  # 是否跳过归一化
):

    # config.ini，拿到 max_len
    cfg = ConfigParser()
    cfg.read(config_path)
    max_len = cfg.getint("construct", "max_len")

    print(f"[construct] 从配置文件中读取到 max_len = {max_len}")


    # 加载 MicroTokenizer.pkl
    tokenizer_path = Path(tokenizer_path)
    if not tokenizer_path.is_file():
        raise FileNotFoundError(f"找不到 tokenizer 文件：{tokenizer_path}")

    with open(tokenizer_path, "rb") as f:
        # 用自定义的 CustomUnpickler 来反序列化，是为了在不同环境下
        # 也能正确找到 MicroTokenizer 类所在的模块。
        unpickler = CustomUnpickler(f)
        tokenizer = unpickler.load()

    # 顺便打印一点信息帮助理解 tokenizer
    try:
        vocab_size = len(tokenizer.vocab)
        print(f"[construct] tokenizer 加载完成，词表大小 = {vocab_size}")
    except Exception:
        print("[construct] tokenizer 已加载（未访问 vocab，可能实现不同）")

    # 打印归一化提示
    if not no_normalize:
        print(
            "Your data will be normalized with the phylogeny mean and std. "
            "If you wish to use your own normalization, please set no_normalize=True."
        )


    #  构建 MicroCorpus
    input_path = Path(input_path)
    if not input_path.is_file():
        raise FileNotFoundError(f"找不到输入丰度表文件：{input_path}")

    corpus = MicroCorpus(
        data_path=str(input_path),
        tokenizer=tokenizer,
        key=key,
        max_len=max_len,
        preprocess=not no_normalize,
    )

    print("[construct] MicroCorpus 已构建完成。")
    try:
        print(f"[construct] 语料库样本数量 = {len(corpus)}")
    except TypeError:
        print("[construct] corpus 不支持 len()，可以自行检查内部实现。")


    # 保存到 output_path
    output_path = Path(output_path)
    with open(output_path, "wb") as f:
        dump(corpus, f)

    print(f"[construct] 语料库已保存到：{output_path.resolve()}")

    # 返回 corpus，方便后续在 Notebook 里分析
    return corpus

## 2. 执行

In [3]:
corpus = construct_simple(
    input_path=str(input_path),
    output_path=str(output_path),
    config_path=str(config_path),
    tokenizer_path=str(tokenizer_path),
    key="genus",
    no_normalize=False,
)

[construct] 从配置文件中读取到 max_len = 512
[construct] tokenizer 加载完成，词表大小 = 1121
Your data will be normalized with the phylogeny mean and std. If you wish to use your own normalization, please set no_normalize=True.
0 samples are dropped for all zeroes


100%|██████████| 55575/55575 [00:07<00:00, 7770.41it/s]


Total 55575 samples.
            Max length is 701.
            Average length is 55.86652271704903.
            Min length is 4.
[construct] MicroCorpus 已构建完成。
[construct] 语料库样本数量 = 55575
[construct] 语料库已保存到：/Users/kiancai/STA24/CWD/STAi/ResMicroDb/data/try2_withCC/abundance_A_subgroup_55575.pkl


## 3. 检查生成的结果结构

In [4]:
if 1:
    import torch
    import pandas as pd
    import numpy as np
    # ================= 配置区域 =================
    # 请确认这是你用来构建 corpus 的原始 CSV 路径
    csv_path = "/Users/kiancai/STA24/CWD/STAi/MiCoGPT/data/try2_withCC/abundance_B_13901.csv"
    # ===========================================

    # --- 1. 基础概览与样本检查 (仅查看最短和最长) ---
    pad_id = corpus.tokenizer.pad_token_id
    real_lengths = (corpus.tokens != pad_id).sum(dim=1)

    targets = [
        ("🟢 最短 (Shortest)", torch.argmin(real_lengths).item()),
        ("🔴 最长 (Longest) ", torch.argmax(real_lengths).item())
    ]

    print(f"📊 [概览] 样本总数: {len(corpus)} | 最短长度: {real_lengths.min()} | 最长长度: {real_lengths.max()}")

    for title, idx in targets:
        ids = corpus[idx]['input_ids']
        valid_len = real_lengths[idx].item()
        
        print(f"\n{'='*40}\n🧐 {title} | Index: {idx} | 长度: {valid_len}/{len(ids)}\n{'-'*40}")
        print(f"{'Pos':<5} {'ID':<6} {'Token'}")
        
        # 智能切片：开头5个 ... 中间(EOS附近) ... 结尾
        indices = sorted(list(set(
            list(range(0, 5)) + 
            list(range(valid_len - 2, valid_len + 3)) + 
            list(range(len(ids) - 2, len(ids)))
        )))

        last_i = -1
        for i in indices:
            if i >= len(ids): continue
            if last_i != -1 and i > last_i + 1:
                print(f"{'...':<5} {'...':<6} ...")
            
            token = corpus.tokenizer.convert_ids_to_tokens(ids[i].item())
            print(f"{i:<5} {ids[i]:<6} {token}")
            last_i = i

    # --- 2. 词表利用率与丢失诊断 ---
    print(f"\n{'='*40}\n🚀 词表利用率诊断\n{'='*40}")

    # 统计当前 Corpus 使用情况
    unique_ids = set(torch.unique(corpus.tokens).tolist())
    all_vocab_ids = set(range(corpus.tokenizer.vocab_size))
    special_ids = set(corpus.tokenizer.convert_tokens_to_ids(['<pad>', '<bos>', '<eos>', '<mask>']))
    target_vocab_ids = all_vocab_ids - special_ids # 理论上应有的菌属ID

    used_vocab = len(unique_ids - special_ids)
    missing_ids = target_vocab_ids - unique_ids
    missing_genera = [corpus.tokenizer.convert_ids_to_tokens(i) for i in missing_ids]

    print(f"💡 词表利用率: {used_vocab}/{len(corpus.tokenizer)} ({used_vocab/len(corpus.tokenizer):.1%})")
    print(f"❓ 未出现的菌属数量: {len(missing_genera)}")

    # 如果有丢失，读取 CSV 进行比对 (使用修正后的正则)
    if len(missing_genera) > 0:
        print(f"\n📖 正在读取 CSV 核对: {csv_path}")
        
        try:
            df_raw = pd.read_csv(csv_path, index_col=0)
            # 关键修正：使用更宽容的正则提取列名，确保能匹配到带特殊字符的菌名
            raw_genera = set(df_raw.columns.str.extract(r'(g__[^;]+)').squeeze().tolist())
            
            not_in_csv = [g for g in missing_genera if g not in raw_genera]
            in_csv_but_zero = [g for g in missing_genera if g in raw_genera]
            
            if not_in_csv:
                print(f"\n❌ [严重] {len(not_in_csv)} 个菌在 CSV 表头中完全找不到:")
                print(f"   (如果数量很多，请检查 corpus.py 中的正则是否已修改为 r'(g__[^;]+)')")
                print(f"   示例: {not_in_csv[:5]} ...")
            
            if in_csv_but_zero:
                print(f"\n📉 [正常] {len(in_csv_but_zero)} 个菌在 CSV 中存在，但在语料库中消失:")
                print(f"   原因: 这些菌在所有样本中的丰度可能均为 0，或者被 Phylogeny 树过滤了。")
                print(f"   示例: {in_csv_but_zero[:5]} ...")
                
        except Exception as e:
            print(f"⚠️ 读取 CSV 失败: {e}")
    else:
        print("✅ 完美！所有菌属都已包含 (利用率符合预期)。")

📊 [概览] 样本总数: 55575 | 最短长度: 4 | 最长长度: 512

🧐 🟢 最短 (Shortest) | Index: 1439 | 长度: 4/512
----------------------------------------
Pos   ID     Token
0     1119   <bos>
1     396    g__Pseudomonas
2     481    g__Streptococcus
3     1120   <eos>
4     0      <pad>
5     0      <pad>
6     0      <pad>
...   ...    ...
510   0      <pad>
511   0      <pad>

🧐 🔴 最长 (Longest)  | Index: 23241 | 长度: 512/512
----------------------------------------
Pos   ID     Token
0     1119   <bos>
1     715    g__Oceanobacillus
2     665    g__Pseudogracilibacillus
3     266    g__Tissierella
4     85     g__Methyloligellaceae
...   ...    ...
510   189    g__Gemella
511   732    g__Fusobacterium

🚀 词表利用率诊断
💡 词表利用率: 1117/1121 (99.6%)
❓ 未出现的菌属数量: 0
✅ 完美！所有菌属都已包含 (利用率符合预期)。


/Users/kiancai/STA24/CWD/STAi/MiCoGPT/MiCoGPT/utils/corpus.py:94: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {'input_ids': torch.tensor(tokens),


In [5]:
if 1:
    import pandas as pd
    from tqdm import tqdm

    # ================= 配置 =================
    save_path = project_root / "data" / "try2_withCC/sample_length_stats_B.csv"
    # =======================================

    print("📊 开始统计每个样本的真实词表长度...")

    results = []

    # 检查 corpus 是否保留了原始数据
    if not hasattr(corpus, 'data') or corpus.data is None:
        print("❌ 错误: corpus 对象中没有找到 .data 属性。")
        print("可能是因为构建时为了省内存执行了 del self.data。")
        print("如果这样，你需要重新加载 CSV 到变量 (例如 df_raw) 并在此处遍历它。")
    else:
        # 遍历 corpus 内部存储的原始 dataframe
        # corpus.data 的索引通常就是 Sample ID
        for sample_id in tqdm(corpus.data.index, desc="Analyzing"):
            
            # 1. 获取该样本的一行数据
            row_data = corpus.data.loc[sample_id]
            
            # 2. 调用内部函数计算长度
            # _convert_to_token 返回: (处理后的token列表, 原始长度)
            # 注意：这个 original_len 是包含 <bos> 和 <eos> 的
            _, original_len = corpus._convert_to_token(row_data)
            
            # 3. 计算截断情况
            max_len = corpus.max_len
            # 实际保留的有效长度 (不含 pad)
            kept_len = min(original_len, max_len)
            # 被丢弃的词数量
            truncated_count = max(0, original_len - max_len)
            
            results.append({
                "Sample_ID": sample_id,
                "Original_Length": original_len,   # 原始真实长度
                "Max_Limit": max_len,              # 设定的上限 (512)
                "Truncated_Tokens": truncated_count, # 被截断丢掉的词数
                "Is_Truncated": truncated_count > 0  # 是否发生了截断
            })

        # 4. 生成 DataFrame 并保存
        df_stats = pd.DataFrame(results)
        df_stats.to_csv(save_path, index=False)

        print(f"\n✅ 统计完成！CSV 已保存到: {save_path}")
        
        # 5. 打印一些摘要信息
        trunc_num = df_stats['Is_Truncated'].sum()
        print(f"\n--- 摘要 ---")
        print(f"总样本数: {len(df_stats)}")
        print(f"被截断的样本数: {trunc_num} ({trunc_num/len(df_stats):.2%})")
        if trunc_num > 0:
            max_loss = df_stats['Truncated_Tokens'].max()
            print(f"最严重的截断: 丢掉了 {max_loss} 个词")
            
        # 看看前几行
        print("\n预览生成的表格:")
        print(df_stats.head())

📊 开始统计每个样本的真实词表长度...


Analyzing: 100%|██████████| 55575/55575 [00:07<00:00, 7548.16it/s]



✅ 统计完成！CSV 已保存到: /Users/kiancai/STA24/CWD/STAi/MiCoGPT/data/try2_withCC/sample_length_stats_B.csv

--- 摘要 ---
总样本数: 55575
被截断的样本数: 45 (0.08%)
最严重的截断: 丢掉了 189 个词

预览生成的表格:
   Sample_ID  Original_Length  Max_Limit  Truncated_Tokens  Is_Truncated
0  CRR768228              132        512                 0         False
1  CRR768229              144        512                 0         False
2  CRR768230              195        512                 0         False
3  CRR768231              198        512                 0         False
4  CRR768232              124        512                 0         False
